In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df[df['toxic']==1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


In [5]:
from tensorflow.keras.layers import TextVectorization

In [6]:
x = df['comment_text']
y = df.iloc[:,2:].values

In [7]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [8]:
max_features = 100000 #Number of words in the vocab

In [9]:
vectorizer = TextVectorization(max_tokens = max_features, output_sequence_length = 1800, output_mode = 'int')

In [10]:
vectorizer.adapt(x.values)

In [11]:
vectorizer('riyaz is doing great!')[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([  1,   9, 272, 275], dtype=int64)>

In [12]:
vectorized_text = vectorizer(x.values)

In [13]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [15]:
dataset.as_numpy_iterator().next()

(array([[  731,     1,   331, ...,     0,     0,     0],
        [    5,    41,   752, ...,     0,     0,     0],
        [   49,     7,    83, ...,     0,     0,     0],
        ...,
        [ 3211,   329,     5, ...,     0,     0,     0],
        [13885,     8,  1490, ...,     0,     0,     0],
        [13221,   286,     8, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [16]:
batch_x, batch_y =  dataset.as_numpy_iterator().next()

In [17]:
batch_x.shape

(16, 1800)

In [18]:
#splitting the data

train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.1))

In [19]:
train_generator = train.as_numpy_iterator()

In [20]:
train_generator.next()

(array([[ 918,  218, 2100, ...,    0,    0,    0],
        [   8,   19, 5991, ...,    0,    0,    0],
        [ 248,  111,  425, ...,    0,    0,    0],
        ...,
        [ 654, 1900,  267, ...,    0,    0,    0],
        [   7,   88,    6, ...,    0,    0,    0],
        [  18,   12, 1848, ...,    0,    0,    0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding


In [22]:
model = Sequential()
#Creating Embedding layer
model.add(Embedding(max_features+1,32))
#
model.add(Bidirectional(LSTM(32, activation = 'tanh')))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [23]:
model.compile(loss= 'BinaryCrossentropy', optimizer = 'Adam')

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          3200032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 6)                 390       
                                                        

In [25]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 7438s 1s/step - loss: 0.0634 - val_loss: 0.0454


In [26]:
input_text= vectorizer('you freaking suck')

In [27]:
model.predict(np.expand_dims(input_text,0))

array([[0.9728266 , 0.21274298, 0.8465202 , 0.03509441, 0.7696843 ,
        0.1979531 ]], dtype=float32)

In [28]:
batch = test.as_numpy_iterator().next()

In [29]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [30]:
model.predict(batch_x)

array([[3.06576490e-03, 1.18413311e-06, 7.94589520e-04, 1.03320690e-05,
        2.37971544e-04, 1.34319067e-04],
       [1.77596807e-02, 1.12330390e-05, 2.70655751e-03, 7.29212334e-05,
        1.46308541e-03, 6.73770905e-04],
       [1.04770064e-03, 2.10258818e-07, 2.97635794e-04, 2.32959042e-06,
        6.83586768e-05, 3.86889733e-05],
       [1.62068009e-03, 6.19899879e-07, 5.39630651e-04, 5.90207901e-06,
        1.34944916e-04, 7.63002754e-05],
       [6.01263165e-01, 1.21857226e-02, 2.56462485e-01, 1.33102238e-02,
        2.72090316e-01, 5.04914224e-02],
       [8.01533461e-04, 1.23909174e-07, 2.23338604e-04, 1.49392963e-06,
        4.84249067e-05, 2.72982252e-05],
       [7.37398863e-04, 1.13792879e-07, 2.11745501e-04, 1.38347070e-06,
        4.48778519e-05, 2.53646940e-05],
       [2.18245387e-03, 8.14230873e-07, 6.29901886e-04, 7.31724685e-06,
        1.71363354e-04, 9.78438547e-05],
       [5.59359789e-04, 5.94502261e-08, 1.51306391e-04, 8.05100228e-07,
        3.01894488e-05, 

### Evaluating the model

In [31]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [32]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [33]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [34]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8653128743171692, Recall:0.6103478670120239, Accuracy:0.451354056596756


In [35]:
model.save('toxicity.h5')

In [36]:
model = tf.keras.models.load_model('toxicity.h5')

In [38]:
input_str = vectorizer('hey i freaken hate you!')
res = model.predict(np.expand_dims(input_str,0))
res

array([[0.6369866 , 0.04211345, 0.3355354 , 0.04101637, 0.35824066,
        0.10254854]], dtype=float32)

In [39]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [41]:
import gradio as gr

In [42]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [43]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://31812.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x23e048a0550>,
 'http://127.0.0.1:7860/',
 'https://31812.gradio.app')